# PROJET 8 : DÉPLOYEZ UN MODÈLE DANS LE CLOUD

## 0 - COMMENTAIRE :

Il m'a été impossible d'installer correctement Pyspark sur mon environnement Windows 10 - Anaconda. Après de nombreux essais, et combinaisons de versions java/python/pyspark, j'ai opté pour l'utilisation de la plateforme Databricks. Le code ci-dessous a été exécuté sous cette plateforme ; la mise en page (dont ce commentaire) a, quant à elle, été effectuée sur Jupyter notebook.

L'environnement Databricks Community Édition impliquait un système de gestion de données particulier qui a notamment énormément compliqué le redimensionnement des images (les bibliothèques PIL ou CV2 ne pouvaient lire les images depuis la source, seul Spark via la commande load.format("image") le permettait. J'ai pu résoudre ce problème (voir II-B) en adaptant un code trouvé dans un article sur le traitement des images avec Spark : https://godatadriven.com/blog/real-distributed-image-processing-with-apache-spark/

N'ayant pu installer Spark sur mon PC, il m'était impossible de lancer un script pyspark via, par exemple, un cluster EMR de Amazon Web Services. J'ai résolu ce problème en souscrivant à l'offre d'essai de Databricks AWS Édition. Ce dernier m'a permis de lier mes comptes Databricks et AWS et d'exécuter ainsi mon code depuis Databricks sur les données stockées sur mon bucket S3.

J'ai enregistré les étapes effectuées pour déployer mon programme de preprocessing avec un enregistreur d'actions utilisateur (l'enregistrement est présent dans les livrables sous le numéro 7)

## I - PRÉALABLES :

### A - Importation des librairies :

In [0]:
# Importation des librairies :
from typing import Iterator
import numpy as np
import pandas as pd
from PIL import Image
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, udf, col
from pyspark.sql.types import StructType, StructField, ArrayType, IntegerType
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.ml.feature import PCA

### B - Définition du répertoire :

#### 1 - Databricks Community Edition :

In [ ]:
# Adresse du répertoire et récupération des catégories :
path = "/FileStore/shared_uploads/histcomsfh@gmail.com/dataset/"
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path))
cat = [file.getPath().getName() for file in list_status]

#### 2 - AWS :

In [0]:
# Adresse du répertoire :
path = "s3a://vivianorsprojet8/dataset/"

### C - Instanciation de Spark :

In [0]:
# Instanciation de Spark :
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

## II - PREPROCESSING DES IMAGES :

### A - Chargement des images :

In [0]:
# Récupération des images :
df = spark.read.format("image").load(path + "*")
print((df.count(), len(df.columns)))
print(df.printSchema())
df.show(5)

(200, 1)
root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

None
+--------------------+
|               image|
+--------------------+
|{dbfs:/FileStore/...|
|{dbfs:/FileStore/...|
|{dbfs:/FileStore/...|
|{dbfs:/FileStore/...|
|{dbfs:/FileStore/...|
+--------------------+
only showing top 5 rows



### B - Traitement et redimensionnement des images :

In [0]:
# Traitement et redimensionnement des images (à cause d'erreurs mémoire lors de la réduction de dimension)
schema = StructType(df.select("image.*").schema.fields + [
    StructField("data_array", ArrayType(IntegerType()), True)
])

def preprocess_array_image(data):
    mode = 'RGB' 
    img = Image.frombytes(mode=mode, data=data.data, size=[100, 100])
    img = img.resize([10, 10])
    arr = np.asarray(img)
    arr = arr.reshape([10*10*3])
    return arr.flatten()

def array_image(dataframe_batch_iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for dataframe_batch in dataframe_batch_iterator:
        dataframe_batch["data_array"] = dataframe_batch.apply(preprocess_array_image, axis=1)
        yield dataframe_batch

df = df.select("image.*").mapInPandas(array_image, schema)

df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|
+--------------------+------+-----+---------+----+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FF FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FD FE FF FD FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...

### C - Vectorisation :

In [0]:
# Vectorisation :
ImageSchema.imageFields
img2vec = udf(lambda l: Vectors.dense(l), VectorUDT())
df = df.withColumn("vectors", img2vec("data_array"))
df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|             vectors|
+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FF FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2

### D - Standardisation :

In [0]:
# Standardisation :
scaler = StandardScaler(inputCol="vectors", outputCol="scaled_vectors", withMean=True, withStd=True)
model_std = scaler.fit(df)
df = model_std.transform(df)
df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|             vectors|      scaled_vectors|
+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF F

### E - Réduction de dimension (PCA) :

In [0]:
# Réduction de dimension :
pca = PCA(k=10, inputCol="scaled_vectors", outputCol="vectors_redux")
redux = pca.fit(df)
df = redux.transform(df)
df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|             vectors|      scaled_vectors|       vectors_redux|
+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|[13.6054451869003...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|[13.3463750962408...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 255, 2...|[255.0,255.0,255....|[0.07071067811898...|[14.2306277378380...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF F

### F - Finalisation du Dataframe :

In [0]:
# Réorganisation du Dataframe et ajout de la target :
df = df.select("origin","vectors_redux")
df = df.withColumn("target", split(col("origin"), "dataset/").getItem(1))
df = df.withColumn("target", split(col("target"), "/").getItem(0))
df.show()

+--------------------+--------------------+---------+
|              origin|       vectors_redux|   target|
+--------------------+--------------------+---------+
|dbfs:/FileStore/s...|[13.6054451869003...|Pineapple|
|dbfs:/FileStore/s...|[13.3463750962408...|Pineapple|
|dbfs:/FileStore/s...|[14.2306277378380...|Pineapple|
|dbfs:/FileStore/s...|[14.3587066936779...|Pineapple|
|dbfs:/FileStore/s...|[13.8885565638878...|Pineapple|
|dbfs:/FileStore/s...|[13.6723987628422...|Pineapple|
|dbfs:/FileStore/s...|[14.2132031079448...|Pineapple|
|dbfs:/FileStore/s...|[13.8688005540647...|Pineapple|
|dbfs:/FileStore/s...|[13.9802638320546...|Pineapple|
|dbfs:/FileStore/s...|[14.912476519928,...|Pineapple|
|dbfs:/FileStore/s...|[13.8943097883129...|Pineapple|
|dbfs:/FileStore/s...|[14.1397887641841...|Pineapple|
|dbfs:/FileStore/s...|[14.2733212182557...|Pineapple|
|dbfs:/FileStore/s...|[14.2099528354214...|Pineapple|
|dbfs:/FileStore/s...|[14.9796009051326...|Pineapple|
|dbfs:/FileStore/s...|[14.30

## III - EXPORT ET ASSEMBLAGE D'UNE MATRICE CSV :

### A - Export aux formats json et parquet :

In [ ]:
# Export :
resultats = "s3://vivianorsprojet8/résultats/"
df.write.parquet(résultats)
df.write.json(résultats)

### B - Réimport, assemblage et export au format csv :

#### 1) Commentaire :

Cette partie III - B a été réalisée non plus sur Databricks, mais sur Jupyter Notebook. Les fichiers json (ou parquet) générés sur le cloud (Databricks + AWS) ont été téléchargés et importés ci-dessous pour créer une matrice csv. Les colonnes "added" et "removed" correspondent à des logs AWS lors de l'opération sur le cloud.

#### 2) Importation, lecture et transformation des fichiers json en une matrice csv :

In [8]:
# Importation de librairies
import pandas as pd
import glob

In [9]:
# Répertoires des fichiers json :
json_path = "C:/Users/7700k/Desktop/json/"

In [10]:
# Fonction de lecture des fichiers json partitionnés :
def readFiles(path):
    files = glob.glob(path + "*")
    dfs = [] # an empty list to store the data frames
    for file in files:
        data = pd.read_json(file, lines=True) # read data frame from json file
        dfs.append(data) # append the data frame to the list

    df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.
    return df

In [11]:
# Lecture des fichiers json réunis :
csv_data = readFiles(json_path)

In [12]:
# Restriction de la colonne vectors sur les valeurs uniquement :
csv_data["vectors_redux"] = pd.json_normalize(csv_data["vectors_redux"])["values"]
csv_data.head()

,origin,vectors_redux,target,added,removed
0,s3a://vivianorsprojet8/dataset/Pineapple/116_1...,"[11.155305263739486, -1.9322724265188072, 0.01...",Pineapple,NaN,NaN
1,s3a://vivianorsprojet8/dataset/Pineapple/119_1...,"[10.836595752156528, -1.647366647527635, -0.50...",Pineapple,NaN,NaN
2,s3a://vivianorsprojet8/dataset/Pineapple/112_1...,"[11.929215624288979, -2.039589383236388, 0.353...",Pineapple,NaN,NaN
3,s3a://vivianorsprojet8/dataset/Pineapple/149_1...,"[11.681932915058056, -1.8990681139259662, -0.8...",Pineapple,NaN,NaN
4,s3a://vivianorsprojet8/dataset/Pineapple/121_1...,"[11.19777385992577, -1.706469556255533, -0.400...",Pineapple,NaN,NaN


#### 3) Export de la matrice :

In [13]:
# Export au format CSV :
csv_data.to_csv("Ors_Vivian_2_images_082022.csv")

## IV - VERS LE DÉPLOIEMENT DE MODÈLES PERFORMANTS :

### A - Commentaire :

La mission de ce projet 8 précisait, je cite "Il n'est pas nécessaire d’entraîner un modèle pour le moment. L’important est de mettre en place les premières briques de traitement qui serviront lorsqu’il faudra passer à l’échelle en termes de volume de données !" Le code Spark déployé sur le cloud transforme respecte cette consigne en transformant les images d'entrée en briques véritablement élémentaires, à savoir en vecteurs soumis à une réduction de dimension.

D'autres méthodes auraient pu être envisagées, telles que l'utilisation de descripteurs (SIFT, SURF...) ou, surtout, de features pour modèle de CNN Transfer Learning tels que VGG16, VGG19 ou ResNet50. Ces méthodes auraient, cependant, nécessité un budget supérieur du fait du transfert de données, de la taille des images en entrée (224 * 224 * 3) et de calculs plus complexes. Or, tous les coûts inhérents étaient à ma charge...

Pour une entreprise qui souhaiterait mettre en place un modèle de reconnaissance d'image, ce type de modèle est évidemment bien plus performant qu'un modèle par vecteurs simples (réduit et redimensionné qui plus est) et doit nécessairement être envisagé dans un avenir proche (et budgétisé).

Auparavant, les opérations de CNN Transfer Learning dans un environnement parallélisé s'effectuaient via la librairie SparkDL et ses modules DeepImageFeaturizer, KerasImageFileTransformer et DeepImagePredictor. Cette librairie est aujourd'hui obsolète ; elle n'est plus mise à jour et ne fonctionne plus avec les dernières librairies Tensorflow. Le deep learning avec Spark s'effectue aujourd'hui principalement avec le module "pandas_udf" qui s'utilise comme un décorateur. On peut en voir un exemple dans l'article sur le preprocessing d'image avec Pyspark : https://godatadriven.com/blog/real-distributed-image-processing-with-apache-spark/

J'ai adapté la méthode des chercheurs qui ont rédigé cet article à notre problématique. Le code qui suit transforme donc nos images en features pour un modèle VGG16. Il a été créé et testé sur Databricks Community Edition puis mis en page sur Jupyter Notebook. Mais encore une fois, et pour des raisons de coûts, il n'a pas été déployé sur AWS et la méthode précédente des vecteurs simples lui a été préférée pour cette opération. 

### B - Préalables :

#### 1) Importation des librairies :

In [0]:
from typing import Iterator
import numpy as np
import pandas as pd
from PIL import Image
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col, split
from pyspark.sql.types import StructType, StructField, ArrayType, IntegerType, FloatType
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.ml.feature import PCA
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

#### 2) Adresse du répertoire :

In [0]:
# Adresse du répertoire :
path = "/FileStore/shared_uploads/histcomsfh@gmail.com/dataset/"

#### 3) Instanciation de Spark :

In [0]:
# Instanciation de Spark :
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

### C - Opération de transformation en features pour VGG16 :

#### 1) Chargement des images :

In [0]:
# Récupération des images :
df = spark.read.format("image").load(path + "*")

#### 2) Traitement et redimensionnement des images au format VGG16 :

In [0]:
# Traitement et redimensionnement des images (à cause d'erreurs mémoire)
schema = StructType(df.select("image.*").schema.fields + [
    StructField("data_array", ArrayType(IntegerType()), True)
])

def preprocess_array_image(data):
    mode = 'RGB' 
    img = Image.frombytes(mode=mode, data=data.data, size=[100, 100])
    img = img.resize([224, 224])
    arr = np.asarray(img)
    arr = arr.reshape([224*224*3])
    return arr

def array_image(dataframe_batch_iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for dataframe_batch in dataframe_batch_iterator:
        dataframe_batch["data_array"] = dataframe_batch.apply(preprocess_array_image, axis=1)
        yield dataframe_batch

df = df.select("image.*").mapInPandas(array_image, schema)

df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|
+--------------------+------+-----+---------+----+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FF FF FF F...|[255, 255, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FD FE FF FD FF F...|[253, 254, 255, 2...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...

#### 3) Transfert de modèle (VGG16) et calcul des features :

In [0]:
# Préparation des images et récupération des features :
def normalize_array(arr):
    return preprocess_input(arr.reshape([224,224,3]))

@pandas_udf(ArrayType(FloatType()))
def featurize(iterator: Iterator[pd.Series]) -> Iterator[pd.Series] :
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    for input_array in iterator :
        normalized_input = np.stack(input_array.map(normalize_array))
        preds = model.predict(normalized_input)
        yield pd.Series(list(preds))
df = df.withColumn("cnn_features", featurize("data_array"))
df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|        cnn_features|
+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[3.4809058E-5, 9....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[4.8657355E-5, 1....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[5.0807892E-5, 9....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[8.080019E-5, 1.8...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FF FF FF F...|[255, 255, 255, 2...|[3.6515834E-5, 5....|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2

#### 4) Vectorisation des features :

In [0]:
# Vectorisation :
ImageSchema.imageFields
img2vec = udf(lambda l: Vectors.dense(l), VectorUDT())
df = df.withColumn("cnn_vectors", img2vec("cnn_features"))
df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|        cnn_features|         cnn_vectors|
+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[3.4809058E-5, 9....|[3.48090579791460...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[4.8657355E-5, 1....|[4.86573553644120...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[5.0807892E-5, 9....|[5.08078919665422...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[8.080019E-5, 1.8...|[8.08001932455226...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF F

#### 5) Réduction de dimension :

In [0]:
# Réduction de dimension :
pca = PCA(k=10, inputCol="cnn_vectors", outputCol="cnn_vectors_redux")
redux = pca.fit(df)
df = redux.transform(df)
df.show()

+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|              origin|height|width|nChannels|mode|                data|          data_array|        cnn_features|         cnn_vectors|   cnn_vectors_redux|
+--------------------+------+-----+---------+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[3.4809058E-5, 9....|[3.48090579791460...|[-0.1024032552965...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[4.8657355E-5, 1....|[4.86573553644120...|[-0.1533783567273...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF FF F...|[255, 255, 254, 2...|[5.0807892E-5, 9....|[5.08078919665422...|[-0.0969303836251...|
|dbfs:/FileStore/s...|   100|  100|        3|  16|[FF FF FE FF F

#### 6) Finalisation du Dataframe :

In [0]:
# Réorganisation du Dataframe et ajout de la target :
df = df.select("origin", "cnn_vectors", "cnn_vectors_redux")
df = df.withColumn("target", split(col("origin"), "dataset/").getItem(1))
df = df.withColumn("target", split(col("target"), "/").getItem(0))
df.show()

+--------------------+--------------------+--------------------+---------+
|              origin|         cnn_vectors|   cnn_vectors_redux|   target|
+--------------------+--------------------+--------------------+---------+
|dbfs:/FileStore/s...|[3.48090579791460...|[-0.1024032552965...|Pineapple|
|dbfs:/FileStore/s...|[4.86573553644120...|[-0.1533783567273...|Pineapple|
|dbfs:/FileStore/s...|[5.08078919665422...|[-0.0969303836251...|Pineapple|
|dbfs:/FileStore/s...|[8.08001932455226...|[-0.1843001580400...|Pineapple|
|dbfs:/FileStore/s...|[3.65158339263871...|[-0.1863476434376...|Pineapple|
|dbfs:/FileStore/s...|[3.82833859475795...|[-0.1301558632758...|Pineapple|
|dbfs:/FileStore/s...|[3.79108605557121...|[-0.2890689602910...|Pineapple|
|dbfs:/FileStore/s...|[5.01877002534456...|[-0.1056316452319...|Pineapple|
|dbfs:/FileStore/s...|[5.02187212987337...|[-0.1519747746637...|Pineapple|
|dbfs:/FileStore/s...|[2.63436922978144...|[-0.1755102466477...|Pineapple|
|dbfs:/FileStore/s...|[3.

### D - Conclusion :

Il est possible d'adapter assez rapidement le code déployé sur le cloud afin de mettre en place un preprocessing pour des modèles performants. Seulement, cela ne peut être fait sans au moins trois étapes préalables :

- Une concertation avec les commanditaires,
- La sélection et le perfectionnement d'un modèle en local afin d'adapter l'étape de preprocessing,
- Et bien évidemment, ce qui a manqué le plus à ce projet : un budget permettant de travailler sereinement sur le cloud.